In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import re

## Step 1 - Scraping

### Nasa Mars News Scraping

In [2]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)
response

<Response [200]>

In [3]:
soup = bs(response.text, "html.parser")
#print(soup.prettify())

In [4]:
news_title = soup.find("div", class_="content_title").a.text.strip('\n')
news_title

'NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities'

In [5]:
news_p = soup.find('div', class_='rollover_description_inner').text.strip('\n')
news_p

'Starting July 27, news activities will cover everything from mission engineering and science to returning samples from Mars to, of course, the launch itself.'

### JPL Mars Space Images - Featured Image

In [6]:
# Using splinter to create Browser
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Luis\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [7]:
browser.visit("https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars")

In [8]:
html = browser.html
soup = bs(html, "html.parser")

In [9]:
base_url = "https://www.jpl.nasa.gov"
image_url = soup.find("article", class_="carousel_item")["style"]
image_url = image_url.lstrip("background-image: url('")
image_url = image_url.rstrip("');")
featured_image_url = base_url + image_url
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23341-1920x1200.jpg'

### Mars Weather

In [10]:
browser.visit("https://twitter.com/marswxreport?lang=en")

In [11]:
html = browser.html
soup = bs(html, "html.parser")

In [12]:
#divs_list = soup.select('div:contains("InSight")')
#for div in divs_list:
#    if "InSight" in div.text:
#        print(div.text)

In [13]:
mars_weather = ""
list_tweets = soup.find_all("div", class_="css-901oao")
for tweet in list_tweets:
    if "InSight" in tweet.text:
        mars_weather = tweet.text
        break
mars_weather

''

### Mars Facts

In [14]:
#browser.visit("https://space-facts.com/mars/")

In [15]:
# <table id="tablepress-p-mars" class="tablepress tablepress-id-p-mars">

In [45]:
url = "https://space-facts.com/mars/"
tables = pd.read_html(url)

In [48]:
html_table = tables[0].copy()
html_table = html_table.rename(columns={0:"Description", 1:"Value"})
html_table = html_table.set_index("Description")
html_table

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [49]:
pd_table_html = html_table.to_html()
pd_table_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

### Mars Hemispheres

In [26]:
# Visit with the browser the main page
browser.visit("https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")

In [27]:
# Parse into html and make a soup object
html = browser.html
soup = bs(html, "html.parser")

In [34]:
# url type: https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
# Base URL to get the full image url 
base_url= "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/"

# Get the div class="description" where the link to each hemisphere is
list_items = soup.find_all("div", class_="description")

hemisphere_image_urls = []

# For each link we get the href to get the full image and the description
for item in list_items:
    aux_str = item.a['href'].split("/")[5]
    hemisphere_image_urls.append(
        {
            "title": item.a.text.rstrip(" Enhanced"),
            "img_url": f"{base_url}{aux_str}.tif/full.jpg"
        }
    )

#hemisphere_image_urls

In [22]:
# create a list where to put the images links
#list_img_url = []
#browser.visit("https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced")
# For each url -> visit the page and get the full url of the image
#for url in list_urls:
    # create full url to visit with the browser
#    desc_url = base_url + url
#    browser.visit(desc_url)
    # get the html and make it a soup object
#    html_desc = browser.html
#    soup_desc = bs(html, "html.parser")
    
    

In [23]:
# get the html and make it a soup object
#html_desc = browser.html
#soup_desc = bs(html, "html.parser")
#img_url = soup.find_all("a")
#img_url = img_url.findChilden()
#print(len(img_url))
#print(img_url)